In [6]:
import sys, os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import ReduceLROnPlateau, TensorBoard, EarlyStopping, ModelCheckpoint
from keras.models import load_model
from keras.models import model_from_json
from keras.models import Model
import hickle as hkl
import tensorflow as tf

In [13]:
num_features = 64
num_labels = 7
batch_size = 1
epochs = 100
width, height = 48, 48

x = hkl.load('./data.hkl').astype(np.float32)#.reshape((48, 48, 1566))
y = hkl.load('./label.hkl')


l = []
for i in range(1566):
    l.append(x[i][0])
x = np.array(l)

x -= np.mean(x, axis=0)
x /= np.std(x, axis=0)

# for xx in range(10):
#    plt.figure(xx)
#    plt.imshow(x[xx].reshape((48, 48)), interpolation='none', cmap='gray')
# plt.show()

#splitting into training, validation and testing data
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=41)

# #saving the test samples to be used later
# np.save('modXtest', X_test)
# np.save('modytest', y_test)

#desinging the CNN
model = Sequential()

model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', input_shape=(width, height, 1), data_format='channels_last', kernel_regularizer=l2(0.01)))
model.add(Conv2D(num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(Conv2D(2*2*2*num_features, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(2*2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*2*num_features, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2*num_features, activation='relu'))
model.add(Dropout(0.5))

model.add(Dense(num_labels, activation='softmax'))

#model.summary()

#Compliling the model with adam optimixer and categorical crossentropy loss
# model.compile(loss=categorical_crossentropy,
#               optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7),
#               metrics=['accuracy'])
model.load_weights('./fer.h5')
model.trainable = False

source_model = model
predictions = Dense(1, activation='sigmoid')(source_model.layers[-2].output)
model = Model(inputs=source_model.input, outputs=predictions)


optimizer = tf.keras.optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-7)
model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])

#training the model
model.fit(np.array(X_train), np.array(y_train),
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(np.array(X_valid), np.array(y_valid)),
          shuffle=True)

# #saving the  model to be used later
# fer_json = model.to_json()
# with open("fer.json", "w") as json_file:
#     json_file.write(fer_json)
# model.save_weights("fer.h5")
# print("Saved model to disk")

Epoch 1/100
317/317 [==============================] - 7s 22ms/step - loss: 1.2506 - accuracy: 0.6663 - val_loss: 0.4733 - val_accuracy: 0.8440
Epoch 2/100
317/317 [==============================] - 7s 22ms/step - loss: 1.0138 - accuracy: 0.7358 - val_loss: 0.4499 - val_accuracy: 0.8440
Epoch 3/100
317/317 [==============================] - 7s 23ms/step - loss: 0.7725 - accuracy: 0.7688 - val_loss: 0.4423 - val_accuracy: 0.8440
Epoch 4/100
317/317 [==============================] - 7s 22ms/step - loss: 0.6852 - accuracy: 0.7946 - val_loss: 0.4253 - val_accuracy: 0.8440
Epoch 5/100
317/317 [==============================] - 7s 22ms/step - loss: 0.7694 - accuracy: 0.7874 - val_loss: 0.4307 - val_accuracy: 0.8440
Epoch 6/100
317/317 [==============================] - 8s 24ms/step - loss: 0.7363 - accuracy: 0.7741 - val_loss: 0.4307 - val_accuracy: 0.8440
Epoch 7/100
317/317 [==============================] - 8s 24ms/step - loss: 0.6208 - accuracy: 0.8042 - val_loss: 0.4371 - val_accuracy:

317/317 [==============================] - 9s 29ms/step - loss: 0.4920 - accuracy: 0.8323 - val_loss: 0.4356 - val_accuracy: 0.8440
Epoch 58/100
317/317 [==============================] - 9s 29ms/step - loss: 0.5063 - accuracy: 0.8126 - val_loss: 0.4311 - val_accuracy: 0.8440
Epoch 59/100
317/317 [==============================] - 9s 29ms/step - loss: 0.4937 - accuracy: 0.8281 - val_loss: 0.4234 - val_accuracy: 0.8440
Epoch 60/100
317/317 [==============================] - 10s 31ms/step - loss: 0.4852 - accuracy: 0.8371 - val_loss: 0.4315 - val_accuracy: 0.8440
Epoch 61/100
317/317 [==============================] - 10s 31ms/step - loss: 0.5185 - accuracy: 0.8149 - val_loss: 0.4275 - val_accuracy: 0.8440
Epoch 62/100
317/317 [==============================] - 9s 28ms/step - loss: 0.4825 - accuracy: 0.8167 - val_loss: 0.4214 - val_accuracy: 0.8440
Epoch 63/100
317/317 [==============================] - 9s 28ms/step - loss: 0.5336 - accuracy: 0.8017 - val_loss: 0.4291 - val_accuracy: 0.8

KeyboardInterrupt: 

In [14]:
#saving the  model to be used later
fer_json = model.to_json()
with open("blow.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("blow.h5")
print("Saved model to disk")

Saved model to disk


In [32]:
model.load_weights('./fer.h5')

In [33]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_32 (Conv2D)           (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_33 (Conv2D)           (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization_28 (Batc (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 23, 23, 64)        0         
_________________________________________________________________
dropout_28 (Dropout)         (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_34 (Conv2D)           (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_29 (Batc (None, 23, 23, 128)      

In [10]:
#saving the  model to be used later
fer_json = model.to_json()
with open("blow.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("blow.h5")
print("Saved model to disk")

Saved model to disk


In [20]:
x[0][0].shape

(48, 48, 1)

In [21]:
l = []
for i in range(1566):
    l.append(x[i][0])
np.array(l)

In [22]:
np.array(l).shape

(1566, 48, 48, 1)

In [34]:
import tensorflow as tf
filename = "blow.json"

with open(filename, "r") as json_file:
    loaded_model_json = json_file.read()
    net = tf.keras.models.model_from_json(loaded_model_json)
    net.load_weights("blow.h5")